<a href="https://colab.research.google.com/github/ermilovim/ml_kaggle/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Воробьевы горы/Нейронные сети/titanic/train.csv')
train['From'] = 1
test = pd.read_csv('/content/drive/MyDrive/Воробьевы горы/Нейронные сети/titanic/test.csv')
test['From'] = 0
all = [train, test]

In [ ]:
for dataframe in all:
    dataframe['Sal'] = dataframe['Name'].str.extract(r' ([A-Za-z]+)\. ')
    dataframe['TickTag'] = dataframe['Ticket'].str.extract(r'([A-Z])')
    dataframe['Deck'] = dataframe['Cabin'].str.extract(r'([A-Z])')
    dataframe['Rel'] = dataframe['SibSp'] + dataframe['Parch']

    # Drop the Unnecessary Data
    dataframe.drop(['Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Find Missing Values
# print(df_train.isna().sum())
# print(df_test.isna().sum())

# Fill Missing Values and Organize the Dataframe
for dataframe in all:
    # Sal
    dataframe['Sal'] = dataframe['Sal'].replace(['Dr', 'Rev', 'Major', 'Col', 'Countess', 'Capt', 'Sir', 'Lady',
                                                 'Dona', 'Don', 'Jonkheer'], 'Others')
    dataframe['Sal'] = dataframe['Sal'].replace(['Ms', 'Mlle'], 'Miss')
    dataframe['Sal'] = dataframe['Sal'].replace('Mme', 'Mrs')
    # Age
    dataframe.loc[(dataframe['Pclass'] == 1) & (dataframe['Sex'] == 'female'), 'Age'] = dataframe[
        (dataframe['Pclass'] == 1) & (dataframe['Sex'] == 'female')]['Age'].fillna(dataframe[
            (dataframe['Pclass'] == 1) & (dataframe['Sex'] == 'female')]['Age'].mean())
    dataframe.loc[(dataframe['Pclass'] == 1) & (dataframe['Sex'] == 'male'), 'Age'] = dataframe[
        (dataframe['Pclass'] == 1) & (dataframe['Sex'] == 'male')]['Age'].fillna(dataframe[
            (dataframe['Pclass'] == 1) & (dataframe['Sex'] == 'male')]['Age'].mean())
    dataframe.loc[(dataframe['Pclass'] == 2) & (dataframe['Sex'] == 'female'), 'Age'] = dataframe[
        (dataframe['Pclass'] == 2) & (dataframe['Sex'] == 'female')]['Age'].fillna(dataframe[
            (dataframe['Pclass'] == 2) & (dataframe['Sex'] == 'female')]['Age'].mean())
    dataframe.loc[(dataframe['Pclass'] == 2) & (dataframe['Sex'] == 'male'), 'Age'] = dataframe[
        (dataframe['Pclass'] == 2) & (dataframe['Sex'] == 'male')]['Age'].fillna(dataframe[
            (dataframe['Pclass'] == 2) & (dataframe['Sex'] == 'male')]['Age'].mean())
    dataframe.loc[(dataframe['Pclass'] == 3) & (dataframe['Sex'] == 'female'), 'Age'] = dataframe[
        (dataframe['Pclass'] == 3) & (dataframe['Sex'] == 'female')]['Age'].fillna(dataframe[
            (dataframe['Pclass'] == 3) & (dataframe['Sex'] == 'female')]['Age'].mean())
    dataframe.loc[(dataframe['Pclass'] == 3) & (dataframe['Sex'] == 'male'), 'Age'] = dataframe[
        (dataframe['Pclass'] == 3) & (dataframe['Sex'] == 'male')]['Age'].fillna(dataframe[
            (dataframe['Pclass'] == 3) & (dataframe['Sex'] == 'male')]['Age'].mean())

    # Embarked
    dataframe['Embarked'] = dataframe['Embarked'].fillna(dataframe['Embarked'].describe().top)

    # Ticket_Tag
    dataframe['TickTag'] = dataframe['TickTag'].fillna('U')

    # Deck
    dataframe['Deck'] = dataframe['Deck'].fillna('U')

    # Fare
    dataframe['Fare'] = dataframe['Fare'].fillna(dataframe['Fare'].mean())

# print(df_train.info())
# print(df_test.info())

# Numeric Features
for dataframe in all:
    # Sex
    sex_dict = {'male': 0, 'female': 1}
    dataframe['Sex'] = dataframe['Sex'].map(sex_dict)

    # Age
    dataframe.loc[(dataframe['Age'] <= 1), 'Age'] = 0
    dataframe.loc[(dataframe['Age'] > 1) & (dataframe['Age'] <= 10), 'Age'] = 1
    dataframe.loc[(dataframe['Age'] > 10) & (dataframe['Age'] <= 18), 'Age'] = 2
    dataframe.loc[(dataframe['Age'] > 18) & (dataframe['Age'] <= 30), 'Age'] = 3
    dataframe.loc[(dataframe['Age'] > 30) & (dataframe['Age'] <= 45), 'Age'] = 4
    dataframe.loc[(dataframe['Age'] > 45) & (dataframe['Age'] <= 60), 'Age'] = 5
    dataframe.loc[(dataframe['Age'] > 60), 'Age'] = 6

    # Fare
    dataframe.loc[(dataframe['Fare'] <= 8), 'Fare'] = 0
    dataframe.loc[(dataframe['Fare'] > 8) & (dataframe['Fare'] <= 15), 'Fare'] = 1
    dataframe.loc[(dataframe['Fare'] > 15) & (dataframe['Fare'] <= 30), 'Fare'] = 2
    dataframe.loc[(dataframe['Fare'] > 30) & (dataframe['Fare'] <= 100), 'Fare'] = 3
    dataframe.loc[(dataframe['Fare'] > 100) & (dataframe['Fare'] <= 150), 'Fare'] = 4
    dataframe.loc[(dataframe['Fare'] > 250), 'Fare'] = 5

    # Embarked
    embarked_dict = {'S': 0, 'C': 1, 'Q': 2}
    dataframe['Embarked'] = dataframe['Embarked'].map(embarked_dict)

    # Sal
    sal_dict = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Others': 4}
    dataframe['Sal'] = dataframe['Sal'].map(sal_dict)

    # TickTag
    tick_tag_dict = {'U': 0, 'P': 1, 'S': 2, 'C': 3, 'A': 4, 'W': 5, 'F': 6, 'L': 7}
    dataframe['TickTag'] = dataframe['TickTag'].map(tick_tag_dict)

    # Deck
    deck_dict = {'U': 0, 'C': 1, 'B': 2, 'D': 3, 'E': 4, 'A': 5, 'F': 6, 'G': 7, 'T': 8}
    dataframe['Deck'] = dataframe['Deck'].map(deck_dict)

    # Age
    dataframe['Age'] = dataframe['Age'].astype('int64')

    # Fare
    dataframe['Fare'] = dataframe['Fare'].astype('int64')

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,From,Sal,TickTag,Deck,Rel
0,1,0,3,0,3,0,0,1,0,4,0,1
1,2,1,1,1,4,3,1,1,2,1,1,1
2,3,1,3,1,3,0,0,1,1,2,0,0
3,4,1,1,1,4,3,0,1,2,0,1,1
4,5,0,3,0,4,1,0,1,0,0,0,0


In [ ]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,From,Sal,TickTag,Deck,Rel
0,892,3,0,4,0,2,0,0.0,NaN,NaN,0
1,893,3,1,5,0,0,0,2.0,NaN,NaN,1
2,894,2,0,6,1,2,0,0.0,NaN,NaN,0
3,895,3,0,3,1,0,0,0.0,NaN,NaN,0
4,896,3,1,3,1,0,0,2.0,NaN,NaN,2


In [ ]:
df.shape

(891, 23)

In [ ]:
df = df.drop('Name', axis=1)
df = df.drop('Ticket', axis=1)
sex = {'male': 0, 'female': 1}
df['Sex'] = df['Sex'].map(sex)

In [ ]:
le = LabelEncoder()
df['Embarked'] = le.fit_transform(df['Embarked'])

In [ ]:
df['Cabin'] = le.fit_transform(df['Cabin'])

In [ ]:
ohe = OneHotEncoder(sparse=False)
embarked = ohe.fit_transform(df['Embarked'].values.reshape(-1, 1))
tmp = pd.DataFrame(embarked,  
                   columns=['emb ' + str(i) for i in range(embarked.shape[1])])
df = pd.concat([df, tmp], axis=1)
#cabin = ohe.fit_transform(df['Cabin'].values.reshape(-1, 1))
#tmp = pd.DataFrame(cabin,  
#                   columns=['cabin ' + str(i) for i in range(cabin.shape[1])])
#df = pd.concat([df, tmp], axis=1)

In [ ]:
encoded_embarked[0]

In [ ]:
df.head()

In [ ]:
X = train.drop('Survived', axis=1)
y = train['Survived']

In [ ]:
cv = KFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
clf = LogisticRegression(random_state=1)
clf.fit(X_train, y_train)
# scores = cross_val_score(clf, X, y, cv=cv, n_jobs=-1)
# print(scores)
print(accuracy_score(y_test, clf.predict(X_test)))

0.8097014925373134


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
clf = LogisticRegression(random_state=1)
clf.fit(X, y) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Воробьевы горы/Нейронные сети/titanic/test.csv')
test_df.head()

In [ ]:
test_df['Embarked'][0] = '0'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test_df = test_df.fillna('0')

In [ ]:
for i in range(len(test_df['Cabin'])):
  test_df['Cabin'][i] = test_df['Cabin'][i][0]
  print(test_df['Cabin'][i])

In [ ]:
test_df = test_df.fillna('0')
test_df = test_df.drop('Name', axis=1)
test_df = test_df.drop('Ticket', axis=1)
sex = {'male': 0, 'female': 1}
test_df['Sex'] = test_df['Sex'].map(sex)

test_df['Embarked'] = le.fit_transform(test_df['Embarked'])
test_df['Cabin'] = le.fit_transform(test_df['Cabin'])
test_embarked = ohe.fit_transform(test_df['Embarked'].values.reshape(-1, 1))
tmp = pd.DataFrame(test_embarked,  
                   columns=['emb ' + str(i) for i in range(test_embarked.shape[1])])
test_df = pd.concat([test_df, tmp], axis=1)
# test_cabin = ohe.fit_transform(test_df['Cabin'].values.reshape(-1, 1))
# tmp = pd.DataFrame(test_cabin,  
#                    columns=['cabin ' + str(i) for i in range(test_cabin.shape[1])])
# test_df = pd.concat([test_df, tmp], axis=1)

In [ ]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,emb 0,emb 1,emb 2,emb 3
0,892,3,0,34.5,0,0,7.8292,0,0,1.0,0.0,0.0,0.0
1,893,3,1,47,1,0,7,0,3,0.0,0.0,0.0,1.0
2,894,2,0,62,0,0,9.6875,0,2,0.0,0.0,1.0,0.0
3,895,3,0,27,0,0,8.6625,0,3,0.0,0.0,0.0,1.0
4,896,3,1,22,1,1,12.2875,0,3,0.0,0.0,0.0,1.0


In [ ]:
X_test = test

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
with open('.../titanic_submission_1.csv', 'w') as file:
  file.write("PassengerId,Survived\n")
  for i in range(len(predictions)):
    print(test['PassengerId'][i], predictions[i])
    file.write(str(test['PassengerId'][i]) + "," + str(predictions[i]) + "\n")